## Семинар 24.09 - про рекурсию и парсинг сайтов

### Рекурсия

In [50]:
#Рекурсивная функция - это функция, которая вызывает сому себя
# f(n) = n*f(n-1) - рекуррентное соотношение


def factorial(n):
    return n * factorial(n-1)

In [ ]:
#что будет?

factorial(1)

In [55]:
def factorial(n):
    ...
    return n * factorial(n-1)

In [56]:
factorial(1)

1

###  Парсинг сайтов

In [ ]:
from urllib.request import urlopen
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

def download_from_the_internet(url):
    try:
        return urlopen(url).read()
    except:
        return None

    
url = 'https://simple.wikipedia.org/wiki/Main_Page'
html = download_from_the_internet(url)
print(html)

In [7]:
from urllib.parse import urldefrag

In [12]:
from bs4 import BeautifulSoup
# документация
# https://www.crummy.com/software/BeautifulSoup/bs4/doc/

In [32]:
parser = BeautifulSoup(html)

In [33]:
parser.title

<title>Wikipedia</title>

In [34]:
parser.title.name

'title'

In [ ]:
# Найти ссылки можно по тегу a
parser.find_all('a')

In [ ]:
# Все ссылки можно получить (из документации)
for link in parser.find_all('a'):
    print(link.get('href'))

Как отфильтровать реальные ссылки?

https://cat.com/list;meow?breed=siberian

https is the scheme (first element of a URL)
cat.com is the netloc (sits between the scheme and path)
/list is the path (between the netloc and params)
meow is the param (sits between path and query)
breed=siberian is the query (between the fragment and params)

In [25]:
from urllib.parse import urlparse

In [29]:
urlparse('https://cat.com/list;meow?breed=siberian')

ParseResult(scheme='https', netloc='cat.com', path='/list', params='meow', query='breed=siberian', fragment='')

### Задание 1 - Рекурсивный обход википедии (6 баллов)

1) Начиная со страницы (уровень рекурсии 0) https://simple.wikipedia.org/wiki/Main_Page вам необходимо распарсить все ссылки на этой странице, выбрать все ссылки, относящиеся к simple.wikipedia.org 
(имеющие netloc == 'simple.wikipedia.org' либо netloc == '' - относительные ссылки на страницу) и сохранить их в виде списка

2) Далее, рекурсивно для каждой из полученных ссылок повторить алгоритм (грузим страницу по ссылке, парсим все ссылки на странице, сохраняем в виде списка), обозначим этот вызов как рекурсия глубины 1.

3) Ваша задача - остановиться на рекурсии глубины 10, в качестве ответа на задачу - нужно вернуть список ссылок, сколько их у вас получилось?

Глобальными переменными пользоваться запрещено, передаем текущий уровень и список(или set) как аргументы функции!!!

Замечание - если вы уже встречали ссылку ранее, проходить по ней еще раз не нужно


4) Задание оформить в виде файла с названием lastname_seminar6.py, в нем должна быть функция, которая принимает на вход ссылку, текущую глубину и уже пройденные ссылки (см ниже пример функции recursive_calls), допустимо использование вспомогательных функций, но запрещены глобальные переменные

#### Что такое относительная ссылка?

In [ ]:
for link in parser.find_all('a'):
    href = link.get('href')
    parsed_href = urlparse(href)
    # проверяем что ссылка куда-то ведет
    if parsed_href.netloc == 'simple.wikipedia.org' and parsed_href.path: 
        print(href, urlparse(href))

In [ ]:
# Тут мы видим относительную ссылку
for link in parser.find_all('a'):
    href = link.get('href')
    parsed_href = urlparse(href)
    
    if not parsed_href.netloc and parsed_href.path:
        print(href, urlparse(href))

Для того, чтобы наша ссылка была чистой, и мы могли понимать, встречали ли мы ее ранее, нам нужно избавиться от дополнительных параметров ссылки (params, query, fragment)



In [ ]:
from urllib.parse import urljoin

start_page = 'https://simple.wikipedia.org/wiki/Main_Page'

for link in parser.find_all('a'):
    href = link.get('href')
    parsed_href = urlparse(href)
    
    if not parsed_href.netloc and parsed_href.path:
        # чистые ссылки
        print(urljoin(start_page, parsed_href.path))

In [ ]:
# Как должен выглядеть ваш метод, решающий задачу?


def recursive_calls(current_link, depth, visited_links):
    if depth >= 10:
        return
    
    links_from_the_page = your_function(current_link)
    
    ...
    
    for new_link in ...:
        recursive_calls(new_link, depth+1, visited_links)

### Про библиотеку selenium

In [59]:
!pip3 install selenium

  Using cached selenium-3.141.0-py2.py3-none-any.whl (904 kB)


In [69]:
!pip3 install webdriver-manager

In [92]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import time


# driver = webdriver.Chrome()

# если не работает - то так
driver = webdriver.Chrome(ChromeDriverManager().install())

driver.get('https://simple.wikipedia.org/wiki/Main_Page')

print(driver.current_url)
#https://selenium-python.readthedocs.io/locating-elements.html
element = driver.find_element_by_partial_link_text('Hermann')
element.click()

search_element = driver.find_element_by_name('search')
search_element.send_keys('test')
search_element.send_keys(webdriver.common.keys.Keys.ENTER)
time.sleep(5)
driver.close()



INFO:WDM:

====== WebDriver manager ======
INFO:WDM:====== WebDriver manager ======
Current google-chrome version is 94.0.4606
INFO:WDM:Current google-chrome version is 94.0.4606
Get LATEST driver version for 94.0.4606
INFO:WDM:Get LATEST driver version for 94.0.4606
Driver [/Users/pogodina-eval/.wdm/drivers/chromedriver/mac64/94.0.4606.41/chromedriver] found in cache
INFO:WDM:Driver [/Users/pogodina-eval/.wdm/drivers/chromedriver/mac64/94.0.4606.41/chromedriver] found in cache


https://simple.wikipedia.org/wiki/Main_Page


### Задание 2 - 4 балла

1) Найти любую статью из предыдущего задания, которую вы добавили на 10-м уровне рекурсии (нужно написать новую функцию find_link_from_10_depth)

2) с помощью библиотеки selenium нужно начиная с главной страницы (https://simple.wikipedia.org/wiki/Main_Page) перейти на эту статью 

это можно сделать двумя способами:
- найти путь (это сделать можно внутри функции find_link_from_10_depth) от главной страницы  до этой статьи и последовательно переходить по страницам
- с помощью input ('search') вбить название этой статьи и перейти на нее

3) задание должно быть оформлено в виде функции selenium_run(start_url, end_url), которая принимает на вход 2 ссылки - стартовая ссылка и финальная ссылка; перед завершением работы программы вы должны убедиться, что ваш driver находится на странице end_url (driver.current_url == end_url)


Замечание 1 - написанные две функции можно сохранять в том же файле
Замечание 2- вы не должны пользоваться информацией о том, что за статья на 10-м уровне (то есть вы не знаете ее название - вам нужно алгоритмически его получить) 

In [93]:
def selenium_run(start_url, end_url, way_from_start=None):
    '''
    way_from_start - необязательный аргумент, актуален если вы восопльзовались способом 'прокликать' ссылки от начала до конца
    '''
    driver = webdriver.Chrome(ChromeDriverManager().install())

    driver.get(start_url)
    
    ...
    # чтобы убедиться что вы там, где надо
    driver.sleep(10)
    if end_url != driver.current_url:
        print('Oh no!')

### Дедлайн по заданиям - 29.09 23:59